# Riddler Classic - [April 12, 2019](https://fivethirtyeight.com/features/how-many-times-a-day-is-a-broken-clock-right/)


## Problem
_On Sunday, the Baylor Lady Bears won the 2019 NCAA women’s basketball championship, and on Monday, the Virginia Cavaliers did the same on the men’s side._

_But what about all of the unsung transitive champions? For example, earlier in the season, Florida State beat Virginia, thereby laying claim to a transitive championship for the Seminoles. And Boston College beat Florida State, claiming one for the Eagles. And IUPUI beat Boston College, and Ball State beat IUPUI, and so on and so on._

_Baylor, meanwhile, only lost once, to Stanford, who lost to five teams, and so on._

_How many transitive national champions were there this season in the women’s and men’s games? Or, maybe more descriptively, how many teams weren’t transitive national champions? You should include tournament losses in your calculations. All of this season’s women’s results are here and all of the men’s results are here._

## Solution
### Possible Extensions of the Riddle
This weeks riddler just requires writing a program that logs all of the transitive champions. Pretty straight forward. Because of this, I had considered possible extensions to the riddle.  One idea was to also track the number of claims a team has to a transitive championship title. For example, if Team #1 and Team #2 were both transitive champions and Team #3 beat both of them, then Team #3 would have 2 claims as a transitive championship.  From there I would see if there were any patterns in the number of transitive championship claims or if its correleated with the strength of a team.  

However, I quickly realized that trying to count the number of transitive champion claims would lead to an infinite regression in the case were Team A and Team B are both transitive champions and they each beat the other.  There are easy ways to avoid this regression but ultimately I decided against doing this analysis.  

Another idea was to see how the number of teams with claims of being a transitive champion increases as they become more removed from the actual national champion but [Jason Ash](https://twitter.com/ashjasont/status/983080622267613190) has already done looked at this so I didn't feel like doing that analysis would be particularly enlightening.  

All of this is to say I was lazy and didn't extend the riddle which is why I have extra respect for the folks like [Allen Jarvis](https://github.com/allenjarvis/riddler_538) last year that extended the riddle in cool ways.

### My Code
Let's use a few standard packages.

In [71]:
import requests
from bs4 import BeautifulSoup
import re
import queue

# Women's data
file_name = 'womens_ncaa_games.txt'
url = "https://www.masseyratings.com/scores.php?s=305973&sub=305973&all=1"

The code below collects the data from [Massey Ratings](https://www.masseyratings.com) and cleans it a little.  

In [72]:
def get_data(file_name):
    try:
        f = open(file_name, 'r')

    except FileNotFoundError:
        page = requests.get(url)

        soup = BeautifulSoup(page.text, 'html.parser')

        # Clean the data a little
        data_txt = str(soup.find('pre')).lstrip('<pre>').rstrip('</pre>')
        data_txt = re.sub('&amp;', '&', data_txt)
        data_txt = re.sub('\n\nGames: [0-9]*\n\n','', data_txt)

        f = open(file_name, 'w', encoding='utf-8')
        f.write(data_txt)
    else:
        f.close()

To help determine the transitive champions, we create a dictionary with each team and all the teams it lost to. We also created a list of all the teams (which will ultimately be converted to a set) to determine the total number of teams.  

In [73]:
def get_transitive_champs(file_name):
    # determine all the teams each team lost to
    team_losses = {}
    all_teams = []
    with open(file_name) as f:
        for l in f:
            losing_team = l[41:65].strip()
            winning_team = l[12:36].strip()
            if losing_team not in team_losses:
                team_losses[losing_team] = [winning_team, ]
            else:
                team_losses[losing_team].append(winning_team)

            # keep a list of all teams.  this will be converted to a set
            all_teams.append(losing_team)
            all_teams.append(winning_team)
    
    # Now we determine the transtive champs with the help of queue
    all_teams = set(all_teams)
    original_champ = 'Virginia'
    transitive_champions = []
    tracking_queue = queue.Queue()

    tracking_queue.put(original_champ)

    while not tracking_queue.empty():
        team = tracking_queue.get()
        if team not in transitive_champions:
            transitive_champions.append(team)
        else:
            continue

        if team in team_losses:
            for t in team_losses[team]: tracking_queue.put(t)
        else:
            continue
    return all_teams, transitive_champions

def print_results(all_teams, transitive_champions):
    print("number of total teams: " + str(len(all_teams)))
    print("number of transitive champions: " + str(len(transitive_champions)))
    print("number of non transitive champions: " + str(len(all_teams)-len(transitive_champions)))
    print("% of teams that are transitive champions: {0}".format(len(transitive_champions)/len(all_teams)))

Let's get the results from the men's side.  

In [74]:
# Men's 2019 data
file_name = 'mens_2019_ncaa_games.txt'
url = "https://www.masseyratings.com/scores.php?s=305972&sub=12801&all=1"

get_data(file_name)
all_teams, transitive_champs = get_transitive_champs(file_name)
print_results(all_teams, transitive_champs)

number of total teams: 1366
number of transitive champions: 1167
number of non transitive champions: 199
% of teams that are transitive champions: 0.8543191800878477


So there are 1167 transitive champions out of a total 1366 total teams on the men's side meaning there are 199 teams that cannot claim to be transitive champions.  This means that ~85% of the teams could claim to be transitive champions. This is consistent with the results from [last year](https://fivethirtyeight.com/features/can-you-find-the-perfect-poker-hand-can-you-shut-this-infernal-box/) where ~87% of the leage could claim to be transitive champions.

In [75]:
# Women's 2019 data
file_name = 'womens_2019_ncaa_games.txt'
url = "https://www.masseyratings.com/scores.php?s=305973&sub=12801&all=1"

get_data(file_name)
all_teams, transitive_champs = get_transitive_champs(file_name)
print_results(all_teams, transitive_champs)

number of total teams: 1333
number of transitive champions: 1164
number of non transitive champions: 169
% of teams that are transitive champions: 0.873218304576144


On the women's side there are 1164 transitive champions out a total of 1333 teams give leaving 169 teams that cannot claim to be transitive champions.  This result is also similar to the men's data from last year.

### Verify Results
To verify, let's quick check that my code agrees with the [results obtained from last year](https://fivethirtyeight.com/features/can-you-find-the-perfect-poker-hand-can-you-shut-this-infernal-box/).  
#### Men's 2018 results

In [76]:
# Men's 2018 data
file_name = 'mens_2018_ncaa_games.txt'
url = "https://www.masseyratings.com/scores.php?s=298892&sub=12801&all=1"

get_data(file_name)
all_teams, transitive_champs = get_transitive_champs(file_name)
print_results(all_teams, transitive_champs)

number of total teams: 1371
number of transitive champions: 1192
number of non transitive champions: 179
% of teams that are transitive champions: 0.8694383661560905


Well, this isn't exactly the same results as last year but it's close.  I'm hoping the reason for the difference is becuase the dataset was updated after last years Riddler solutions were posted :) The data on Massey does say it was generated on April 13, 2018 which means it could have been updated after the Riddler solutions were posted.  

### Final Observations
__Warning:  The following are my own highly speculative (and probably foolish) ramblings.__

Interestingly, the percentage of teams that can claim to be transitive champions seems to be steady around 85-87%. It would be interesting to investigate what these results are sensitive to. Likely candidates would include the number of games played by each team in a season, the variability of each team's opponents (i.e. each team plays most of it's teams against conference opponents.  I suspect the results would change if teams were assigned opponents randomly), and the likelihood that the best team wins in any given game.  

It might not be too hard to generate synthetic data to investigate the impact of these variables although one might wonder what the value would be in doing that. On that note, do the results of this riddle have any appicability beyond being a fun interesting question to play with? Would there be a compelling reason to generate that synthetic data and study this problem further?

Perhpas this kind of analysis might be relatable to studying network vulerability. We could imagine the 'actual champion' as a node in a network we want access to (or want to protect) and the 'transitive champions' as those nodes that have access to the 'actual champion' node. Each 'transitive champion' node would then have access to the 'actual champion' node via some chain of other 'transitive champion' nodes.  By designing a system that minimizes the number of 'transitive champion' nodes, we could help identify computer network structures that are safer. 